##  Extract, Transform, Load (ETL)



-   Read CSV and JSON file types.
-   Extract data from the above file types.
-   Transform data.
-   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.

In [ ]:
#Import the required modules and functions

In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [7]:

#download a ZIP file from the  URL 

#downloading a ZIP file named "source.zip" from the URL and saving it to the current working directory.
!curl -O  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  4249  100  4249    0     0   1835      0  0:00:02  0:00:02 --:--:--  1837


In [2]:
#Extract zipfile named "datasource.zip" into a directory called "dealership_data".
# The "-d" option specifies the destination directory for the extracted files.

!unzip datasource.zip -d dealership_data


Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


# THE DATA

The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.

In [3]:
#SET PATH

tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [11]:
# Add the CSV extract function below
# Reads a CSV file and stores it in a dataframe variable
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    # Returns the dataframe variable
    return dataframe

In [12]:

# Add the JSON extract function below

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

#The 'lines=True' parameter tells pandas to read the file as a series of JSON objects rather than a single JSON object.



In [ ]:
#xml functios

# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.



In [4]:
def extract_from_xml(file_to_process):
    # Create an empty dataframe with four columns
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    # Use ElementTree to parse the XML file and get the root element
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    # Loop through each person element in the XML file
    for person in root:
        # Get the values of the car_model, year_of_manufacture, price, and fuel elements for the current person
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        # Append a new row to the dataframe with the values for the current person
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    # Return the dataframe containing the XML file data
    return dataframe


In [13]:
#the function will extract data from all the CSV, JSON and XML files in the 'dealership_data' directory 
# and combine the data into a single pandas DataFrame named 'extracted_data'.

def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [14]:
#the 'price' column of a pandas DataFrame named 'data' by rounding it to 2 decimal places. The transformed DataFrame will be returned.

def transform(data):
        data['price'] = round(data.price, 2)
        return data

In [7]:
#This function will write a given pandas DataFrame named 'data_to_load' to a CSV file named 'targetfile'. The function uses the 'to_csv' method of pandas DataFrame to write the data to a CSV file
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile) 

In [8]:
#write message to a log file named 'dealership_logfile.txt'.
#The message will be timestamped with the current time and date using the format specified in the 'timestamp_format' variable. 
# The function will append the message to the end of the log file.
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

In [10]:

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

NameError: name 'extract_from_csv' is not defined

In [15]:
# Log that you have started the ETL process

log("ETL Job Started")

# Log that you have started the Extract step
log("Extract phase Started")
# Call the Extract function
extracted_data = extract()
# Log that you have completed the Extract step
log("Extract phase Ended")
# Log that you have started the Transform step
log("Transform phase Started")
# Call the Transform function
transformed_data = transform(extracted_data)
# Log that you have completed the Transform step

log("Transform phase Ended")
# Log that you have started the Load step
log("Load phase Started")
# Call the Load function
load(targetfile,transformed_data)

# Log that you have completed the Load step

log("Load phase Ended")

# Log that you have completed the ETL process
log("ETL Job Ended")

C:\Users\lasis\AppData\Local\Temp\ipykernel_27764\3260339105.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\lasis\AppData\Local\Temp\ipykernel_27764\3260339105.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\lasis\AppData\Local\Temp\ipykernel_27764\3260339105.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\lasis\AppData\Local\Temp\ipykernel_27764\3260339105.py:13: FutureWarning: The frame.append method is deprecated and will be remove

In [16]:
extracted_data

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,camry,2006,3731.34,Petrol
86,land cruiser,2010,52238.81,Diesel
87,corolla altis,2012,8805.97,Petrol
88,etios liva,2013,5149.25,Petrol


In [17]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
85,camry,2006,3731.34,Petrol
86,land cruiser,2010,52238.81,Diesel
87,corolla altis,2012,8805.97,Petrol
88,etios liva,2013,5149.25,Petrol
